In [1]:
import numpy as np
import pandas as pd

import poissonlearning as pl
import graphlearning as gl

import storage

%load_ext autoreload
%autoreload 2

In [8]:
dataset = "fashionmnist"
dataset_metric = "vae"
experiments = storage.load_results(name="real_data", folder="../results")

In [9]:
p_all = list(set((ex["p"] for ex in experiments if ex["dataset"] == dataset and ex["dataset_metric"] == dataset_metric)))
results_table = pd.DataFrame()
for p in p_all:
    labels_per_class_all = list(set((ex["labels_per_class"] for ex in experiments if np.isclose(ex["p"], p) and ex["dataset"] == dataset and ex["dataset_metric"] == dataset_metric)))
    results_p = pd.Series(index=labels_per_class_all, name=p, dtype="object")
    for num_labels in labels_per_class_all:
        selected_experiments = list(
            filter(
                lambda x: 
                    np.isclose(x["p"], p) 
                    and np.all(np.isclose(x["labels_per_class"], num_labels))
                    and x["dataset"] == dataset,
                experiments
            )
        )
        def _compute_accuracy(experiment):
            prob = experiment["solution"].drop(columns=["x", "y", "true_labels"]).to_numpy()
            scores = prob - np.min(prob)
            scores = scores / np.max(scores)

            # Check if scores are similarity or distance
            pred_labels = np.argmax(scores, axis=1)
            accuracy = gl.ssl.ssl_accuracy(experiment["solution"]["true_labels"], pred_labels, experiment["labels_per_class"] * 10)
            return accuracy
        accuracy = [_compute_accuracy(ex) for ex in selected_experiments]
        accuracy_mean = np.mean(accuracy)
        accuracy_std = np.std(accuracy)
        results_p[num_labels] = f"{accuracy_mean:.3f} ({accuracy_std:.2f})"

    def _extend_results(results_table, new_entries):
        new_index = np.union1d(results_table.index, new_entries.index)
        results_table = results_table.reindex(new_index)
        new_entries = new_entries.reindex(new_index)
        results_table[new_entries.name] = new_entries
        return results_table

    results_table = _extend_results(results_table, results_p)
results_table

,2.0,3.0,4.0,5.0,6.0,8.0
1,58.312 (0.00),55.680 (0.00),53.219 (0.00),51.817 (0.00),50.901 (0.00),49.685 (0.00)
2,67.521 (0.00),63.911 (0.00),62.307 (0.00),61.160 (0.00),60.244 (0.00),58.610 (0.00)
5,66.201 (0.00),64.878 (0.00),63.540 (0.00),62.417 (0.00),61.784 (0.00),61.223 (0.00)
10,69.522 (0.00),68.174 (0.00),67.565 (0.00),67.101 (0.00),66.957 (0.00),66.580 (0.00)


In [11]:
print(results_table.T.to_latex())

\begin{tabular}{lllll}
\toprule
{} &             1  &             2  &             5  &             10 \\
\midrule
2.0 &  58.312 (0.00) &  67.521 (0.00) &  66.201 (0.00) &  69.522 (0.00) \\
3.0 &  55.680 (0.00) &  63.911 (0.00) &  64.878 (0.00) &  68.174 (0.00) \\
4.0 &  53.219 (0.00) &  62.307 (0.00) &  63.540 (0.00) &  67.565 (0.00) \\
5.0 &  51.817 (0.00) &  61.160 (0.00) &  62.417 (0.00) &  67.101 (0.00) \\
6.0 &  50.901 (0.00) &  60.244 (0.00) &  61.784 (0.00) &  66.957 (0.00) \\
8.0 &  49.685 (0.00) &  58.610 (0.00) &  61.223 (0.00) &  66.580 (0.00) \\
\bottomrule
\end{tabular}

